<a href="https://colab.research.google.com/github/yceffort/pytorch-study/blob/master/02)_multi_layer_perception_(1)_wine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip3 install https://download.pytorch.org/whl/cu80/torch-1.0.0-cp36-cp36m-linux_x86_64.whl
!pip3 install torchvision

In [0]:
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split

import pandas as pd

In [0]:
wine = load_wine()
wine_data = wine.data[0:130]
wine_target = wine.target[0:130]

train_X, test_X, train_Y, test_Y = train_test_split(wine_data, wine_target, test_size=0.2)

In [0]:
# 데이터를 텐서 형태로 변환
train_X = torch.from_numpy(train_X).float()
train_Y = torch.from_numpy(train_Y).long()

test_X = torch.from_numpy(test_X).float()
test_Y = torch.from_numpy(test_Y).float()

In [0]:
train = TensorDataset(train_X, train_Y)
train_loader = DataLoader(train, batch_size=16, shuffle=True)

In [0]:
class Net(nn.Module):
  
  def __init__(self):
    super(Net, self).__init__()
    self.fc1 = nn.Linear(13, 96)
    self.fc2 = nn.Linear(96, 96)
    self.fc3 = nn.Linear(96, 96)
    self.fc4 = nn.Linear(96, 96)
    self.fc5 = nn.Linear(96, 96)
    self.fc6 = nn.Linear(96, 2)
    
    
  def forward(self, x):
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = F.relu(self.fc3(x))
    x = F.relu(self.fc4(x))
    x = F.relu(self.fc5(x))

    x = self.fc6(x)
    
    return F.log_softmax(x)
  
  
model = Net()

In [19]:
#오차함수
criterion = nn.CrossEntropyLoss()

#최적화 담당
optimizer = optim.SGD(model.parameters(), lr=0.01)

#학습 300회 ㄱㄱ
for epoch in range(300):
  total_loss = 0
  
  for train_x, train_y in train_loader:
    train_x, train_y = Variable(train_x), Variable(train_y)
    
    #경사 초기화
    optimizer.zero_grad()
    
    #순전파
    output = model(train_x)
    
    #오차
    loss = criterion(output, train_y)
    
    #역전파 계산
    loss.backward()
    
    #가중치 업데이트
    
    optimizer.step()
    
    #총 오차 업데이트
    total_loss += loss.data.item()
    
   
  if (epoch + 1) % 50 == 0:
    print(epoch + 1, total_loss)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:22: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


50 2.2159287333488464
100 1.2211463805288076
150 1.2938374131917953
200 1.3205499947071075
250 1.6236471086740494
300 1.3200164809823036


In [20]:
test_x, test_y = Variable(test_X), Variable(test_Y)
result = torch.max(model(test_x).data, 1)[1]
accuracy = sum(test_y.data.numpy() == result.numpy()) / len(test_y.data.numpy())
accuracy

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:22: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


0.8846153846153846